# Set Up

Note: af_dir and disk_data_dir are modified.

In [15]:
import numpy as np
import pandas as pd
import os
import math # required for prediction conversion

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

af_dir = '../../all_faces_bucket/'
disk_data_dir = '../../forensics_split/'
!pip install git+https://github.com/qubvel/efficientnet
import efficientnet.tfkeras
# !sudo kill -9 PID # clear GPU memory where 9 is PID number
# !watch -n0.1 nvidia-smi

model_to_load = 'config8'

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-v1cudxij
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-v1cudxij
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=18397 sha256=3f4686dc6c66908e8ad80716bc6279717580da3016c288e28a6d0fb4f3b8abb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-sv83pi4m/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet


# Functions

**Note: Save time by only running the cells with functions but leaving the others -- they are there for you to see the intermediate steps.**

## Predict Labels for Test Images

In [16]:
# Need to load model to CPU if GPU is busy training
# with tf.device('/cpu:0'):
#     loaded_model = load_model(af_dir + 'trained_models/saved_models/' + architecture + '_model.h5')

def get_model(model):
    '''Loads one of the saved models based on specified architecture
    (vgg, resnet50, mobilenet or xception)'''
    
    return load_model(af_dir + 'trained_models/saved_models/' + model + '.h5')

def get_multidim_predictions(model):
    '''Takes in a loaded model and outputs filenames and multi-dimensional
    predictions for each class.
    
    Works by initiating an instance of ImageDataGenerator which is used for
    flow_from_directory method.'''
    # normalise and centre test data
    datagen = ImageDataGenerator(samplewise_std_normalization=True, samplewise_center=True)
    generator = datagen.flow_from_directory(disk_data_dir + 'validation', target_size=(224, 224),
                                            shuffle = False, batch_size=1)
    filenames = generator.filenames
    nb_samples = len(filenames)
    generator.reset() # figure out this 
    predictions = model.predict(generator, steps = nb_samples, verbose=1, workers=8)
    
    return filenames, predictions

In [3]:
def get_image_predictions(arr, soft=True):
    '''Obtains image predictions.
    soft: a true value returns probabilities as opposed to hard predictions.'''

    if soft:
        # probability of belonging to fake (second) class,
        # hence return second value for each element in the list
        return [el[1] for el in arr]
    # returns a list of 0's and 1's
    return np.argmax(arr, axis=1)

## Building Information Storage

In [4]:
def build_dataframe(filenames):
    index = range(len(filenames))
    df = pd.DataFrame(index = index, columns = ['method', 'video', 'image', 'test/train', 'true label',
                                                  'probability', 'predicted label', 'acc'])
    df = df.fillna(0)
    methods = [el[el.find('/')+1: el.find('_')] for el in filenames]
    video_numbers = [el[el.find('_')+1: el.rfind('_')] for el in filenames]
    # video_numbers = [re.search("_(.*?)\_", el).group(1) for el in filenames] # older version -- does not include second video name for fake videos
    image_numbers = [el[el.find('_')+1: el.find('.')][4:] for el in filenames]
    true_labels = [0 if el[0] == 'a' else 1 for el in filenames]
    
    df['method'] = methods
    df['video'] =  video_numbers
    df['image'] =  image_numbers
    df['true label'] = true_labels
    df['test/train'] = ['test']*len(filenames)
    df['probability'] = predictions
    df['predicted label'] = ['-']*len(filenames)
    df['acc'] = ['-']*len(filenames)
    
    return df

## Conversion to Video Predictions

Next cell contains three functions which are all options for combining image predictions and converting those into a single video prediction. Fraction method considers only a fraction of higest image predictions (useful when only a fraction of video has been manipulated), ConfidentStrategy takes a mean of a subset of predictions depending on popularity of each of the two classes, and Transform makes each prediction more extreme (pushed towards 0.0 or 1.0 depending on its original value) before taking a mean.

In [5]:
def get_mean_of_a_fraction(lst, fraction = 1.0):
    '''Takes in a list and outputs a mean of the fraction of the largest
    elements for that list (by default (fraction is 1) == consider all predictions)
        
    if fraction equals to 1, the output is simply a mean.
        
    This mimics considering only the top third highest probabilities for
    images for a given video to classify that video. The main idea is
    that if a given video has only a fraction of it being manipulated
    (unknown information) then it's likely to be wrongly classfied as original
    if we average all associated probabilities, however, if we take only a
    certain number of highest proabilities that will be much more representative
    and overall robust.'''
        
    sorted_lst = sorted(lst)[::-1] if type(lst) == list else [lst]
    sliced_lst = sorted_lst[0:math.ceil(len(lst)*fraction)] if fraction != (1 or 1.0) else sorted_lst
    return np.mean(sliced_lst)

def get_mean_with_confident_strategy(lst, fraction = 0.75, t = 0.5):
    '''Confident strategy is implemented from first-place solution in DFDC.
    
    The main idea is that if there are a lot of predictions for one class,
    then the average is taken of those predictions only. If that's not the
    case, then a simple mean is outputted.
    
    Inputs:
    1. list of predictions (converted to a list if it's a single prediction)
    2. fraction -- (between 0 and 1) what fraction of the list should predict
    the same class for other predictions to be disregarded when taking a mean
    3. t -- threshold cutoff value between two classes (note: whole notebook
    is structured for a binary classification problem only)'''
    
    lst = np.array(lst)
    num_pred = len(lst)
    num_fakes = np.count_nonzero(lst >= t)
    num_authentic = num_pred - num_fakes

    # if number of fakes is more that 75% of all predictions 
    if num_fakes > int(num_pred * fraction):
        # take predictions which are greater than threshold and average them
        return np.mean(lst[lst >= t])

    # else if number of predictions below threshold value t is more that 75%
    # of all predictions
    elif num_authentic > int(num_pred * fraction):
        # take these predictions and return their mean
        return np.mean(lst[lst < t])
  
    else: # simple mean
        return np.mean(lst)
    
def get_mean_of_transformed_predictions(lst):
    '''Takes a list of predictions, transforms them by individually
    pushing the values away from the centre (0.5) closer towards the
    extremes (0.0 and 1.0). The visualisation is included below.
    
    Returns a mean of transformed predictions.'''

    if type(lst) != list: lst = [lst]
    weights = np.power([abs(el -0.5) for el in lst], 1.0) + 1e-4
    return float((lst * weights).sum() / weights.sum())

In [6]:
def transform(prob, t = 0.15):
    if prob == 0.5:
        return 0.5
    elif prob <= t:
        return 0.0
    elif prob >= (1-t):
        return 1.0
    elif prob > t and prob < 0.5:
        return 0.5 - np.power(abs(prob-0.5), 0.65)
    elif prob < (1-t) and prob > 0.5:
        return 0.5 + np.power(abs(prob-0.5), 0.65)

In [7]:
def convert_predictions(df, threshold = 0.5, option = None, fraction = 0.33):
    ''' Takes in a dataframe, regroups by videos (collecting all predictions
    for a video in one nested list) then (optionally modifies by one of the three
    methods) returning a mean prediction for each video. Lastly, an accuracy
    column is filled by comparing a true label with the predicted one (a mean
    probability is convered into a label subject to threshold value).
    
    Inputs:
    1. df -- dataframe
    2. threshold -- cutoff probability value between classes (by default 0.5)
    3. option -- (by default None) choices are 'transform', 'confident strategy'
    or 'fraction'; correspond to possible list manipulations
    
    Note: if you feed option = 'fraction', then you need to also specify fraction
    value (1.0 means a simple mean, 0.33 means taking top third, et cetera)
    
    if option is not speficied or not among choices then a simple mean is calculated
    4. fraction (= 0.33) -- value for 'fraction' option '''

    # regroup based on method, video title, and test/train category
    df = df.groupby(['method', 'video','test/train', 'true label', 'predicted label'])\
                    ['probability'].apply(list).reset_index()
    
    collected_labels_pred = list(df['probability']) # get the nested list
    
    # next, we apply one of the three methods to get means
    
    if option == 'transform':
        mean_labels_pred = [get_mean_of_transformed_predictions(el) for el in collected_labels_pred]
        
    elif option == 'confident strategy':
        mean_labels_pred = [get_mean_with_confident_strategy(el) for el in collected_labels_pred]
    
    elif option == 'fraction':
        mean_labels_pred = [get_mean_of_a_fraction(el, fraction) for el in collected_labels_pred]
        
    else: # if no option is chosen (or not from the list), output a simple mean per video
        mean_labels_pred = [np.mean(el) for el in collected_labels_pred]

    labels_pred = [0 if el <= threshold else 1 for el in mean_labels_pred]
    df['predicted label'] = labels_pred

    # produce accuraacy values for each video (0 if classification is wrong and
    # 1 if classicification is correct)
    df['acc'] = [1 if df['true label'][i] == df['predicted label'][i]
                            else 0 for i in range(len(df['true label']))]

    return df

In [8]:
def show_accuracy(df):
    acc_per_method = df.groupby(['test/train', 'method'])['acc'].mean()
    acc_total = df.groupby(['test/train'])['acc'].mean()
    #display(acc_per_method)
    #display(acc_total)
    return acc_per_method, acc_total

# Accuracies

In [17]:
all_config_nums = [2025, 2026, 2027, 2028, 2029, 2030]

for num in all_config_nums:
    
    if num == 2025: aug = 'GridMask'
    elif num == 2026: aug = 'GridMask-Alternative'
    elif num == 2027: aug = 'FaceMask'
    elif num == 2028: aug = 'FacialArtifacts'
    elif num == 2029: aug = 'FacialArtifacts-Alternative'
    elif num == 2030: aug = 'No information-dropping augmentation'
    print(aug)
    loaded_model = get_model('config' + str(num))
    filenames, multidim_predictions = get_multidim_predictions(loaded_model)
    predictions = get_image_predictions(multidim_predictions, soft=True)
    data = build_dataframe(filenames)
    #print('\n')
    for option in ['transform', 'confident strategy', 'fraction', 'None']:
        #print('#'*38)
        new = convert_predictions(data, option = option)
        acc_per_method, acc_total = show_accuracy(new)
        print(option + ':', "%.6f" % acc_total['test'])
        #print('#'*38)
        #if option != 'None': print('\n')
    print('\n')

GridMask
Found 21291 images belonging to 2 classes.
21291/21291 [==============================] - 129s 6ms/step
transform: 0.991429
confident strategy: 0.991429
fraction: 0.991429
None: 0.991429


GridMask-Alternative
Found 21291 images belonging to 2 classes.
21291/21291 [==============================] - 130s 6ms/step
transform: 0.992857
confident strategy: 0.992857
fraction: 0.992857
None: 0.992857


FaceMask
Found 21291 images belonging to 2 classes.
21291/21291 [==============================] - 134s 6ms/step
transform: 0.992857
confident strategy: 0.992857
fraction: 0.990000
None: 0.992857


FacialArtifacts
Found 21291 images belonging to 2 classes.
21291/21291 [==============================] - 133s 6ms/step
transform: 0.991429
confident strategy: 0.991429
fraction: 0.990000
None: 0.991429


FacialArtifacts-Alternative
Found 21291 images belonging to 2 classes.
21291/21291 [==============================] - 134s 6ms/step
transform: 0.992857
confident strategy: 0.992857
fraction